In [1]:
import tensorflow as tf
from tensorflow import keras

from nn.rolr import dataset, model
from physics.hstar import gghzz, c6
from physics.simulation import msq
from physics.hzz import zpair, angles

import os
import json
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler

2024-12-12 12:33:28.609937: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-12 12:33:28.681001: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-12 12:33:28.681025: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-12 12:33:28.681051: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-12 12:33:28.693924: I tensorflow/core/platform/cpu_feature_g

In [2]:
SEED=373485

BASE_SIZE=10000

OUTPUT_DIR='jobs/rolr/first-tests'
CONF_FILE='rolr-config.json'
SCALER_CONF='scaler.json'
SAMPLE_DIR='..'

In [3]:
with open(os.path.join(OUTPUT_DIR, CONF_FILE), 'r') as conf_file:
    config = json.load(conf_file)

In [4]:
sample_train = gghzz.Process(msq.Component.SIG, (0.1, os.path.join(SAMPLE_DIR, 'ggZZ2e2m_sig.csv'), 1.2*BASE_SIZE))
sample_val = gghzz.Process(msq.Component.SIG, (0.1, os.path.join(SAMPLE_DIR, 'ggZZ2e2m_sig.csv'), config['num_events'] + 1.2*BASE_SIZE))
sample_val.events = sample_val.events[config['num_events']:]

In [5]:
z_chooser = zpair.ZPairChooser(bounds1=(70,115), bounds2=(70,115), algorithm='leastsquare')
kin_vars_train = angles.calculate_2(*sample_train.events.filter(z_chooser))
kin_vars_val = angles.calculate_2(*sample_val.events.filter(z_chooser))

sample_train.events = sample_train.events[:BASE_SIZE]
kin_vars_train = kin_vars_train[:BASE_SIZE]
sample_val.events = sample_val.events[:BASE_SIZE]
kin_vars_val = kin_vars_val[:BASE_SIZE]

In [6]:
c6_values = np.linspace(-20,20,11)

c6_mod_train = c6.Modifier(baseline = msq.Component.SBI, sample=sample_train, c6_values = [-5,-1,0,1,5])
c6_weights_train, c6_prob_train = c6_mod_train.modify(c6=c6_values)

c6_mod_val = c6.Modifier(baseline = msq.Component.SBI, sample=sample_train, c6_values = [-5,-1,0,1,5])
c6_weights_val, c6_prob_val = c6_mod_val.modify(c6=c6_values)

In [7]:
train_set = dataset.build_dataset(x_arr=kin_vars_train, 
                                  param_values=c6_values, 
                                  signal_probabilities=c6_weights_train, 
                                  background_probabilities=sample_train.events.weights)
val_set = dataset.build_dataset(x_arr=kin_vars_val, 
                                param_values=c6_values, 
                                signal_probabilities=c6_weights_val, 
                                background_probabilities=sample_val.events.weights)

2024-12-12 12:33:55.761095: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [8]:
with open(os.path.join(OUTPUT_DIR, SCALER_CONF), 'r') as conf_file:
    scaler_conf = json.load(conf_file)

In [9]:
scaler = MinMaxScaler()
scaler.min_ = scaler_conf['scaler.min_']
scaler.scale_ = scaler_conf['scaler.scale_']

train_set = tf.concat([scaler.transform(train_set[:,:10]), train_set[:,10:]], axis=1)
val_set = tf.concat([scaler.transform(val_set[:,:10]), val_set[:,10:]], axis=1)

In [11]:
rolr_model = model.load(os.path.join(OUTPUT_DIR, 'checkpoint.model.tf'))

In [12]:
predictions = rolr_model.predict(train_set[:,:10])
labels = train_set[:,10]

  31/3438 [..............................] - ETA: 18:34

KeyboardInterrupt: 